### 셀레니움(Selenium)이란?
- Selenium은 주로 웹앱을 테스트하는데 사용하는 프레임워크
- webdriver라는 API를 통해 운영체제에 설치된 크롬 등의 브라우저를 제어
- Selenium 모듈 설치 후 사용
- 사용자 브라우저(Chrome, Edge, ..)에 맞는 webdriver를 다운로드 후 사용 가능
- 다운로드 사이트에서 본인이 사용하는 브라우저의 버전등을 확인 후 다운로드
- 크롬: https://chromedriver.chromium.org/downloads
- Edge: https://developer.microsoft.com/en-us/microsoft-edge/tools/webdriver/
- 파이어폭스:https://github.com/mozilla/geckodriver/releases 

#### requests를 이용한 멜론차트 크롤링99999998

In [12]:
# requests를 이용한 멜론차트 크롤링
import requests

url='https://www.melon.com/chart/day/index.html'

html=requests.get(url)
html.status_code

406

In [ ]:
#!pip install selenium

#### selenium을 이용한 멜론차트 크롤링

In [17]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import pandas as pd
import time

import warnings
warnings.filterwarnings(action='ignore')

url='https://www.melon.com/chart/day/index.htm'

browser=webdriver.Chrome('./chromdriver.exe') # 크롬 드라이버 실행
time.sleep(1) # 브라우저 실행시간 대기
browser.get(url) # 실행된 브라우저를 이용해 사이트 이동
browser.implicitly_wait(10) # url 접속 대기(10초) / 접속시 자동으로 다음 코드 이동

In [18]:
html=browser.page_source # 브라주어의 페이지 소스를 모두 가져옴
soup=bs(html, 'html.parser')

browser.quit() # 사용자 브라우저 종료

### 순위정보 가져오기

In [20]:
body_soup=soup.select('table tbody tr')
body_soup

[<tr class="lst50" data-song-no="35945927" id="lst50">
 <td><div class="wrap t_right"><input class="input_check" name="input_check" title="Ditto 곡 선택" type="checkbox" value="35945927"/></div></td>
 <td><div class="wrap t_center"><span class="rank">1</span><span class="none">위</span></div></td>
 <!-- 차트순위 추가 -->
 <td><div class="wrap">
 <span class="rank_wrap" title="순위 동일">
 <span class="bullet_icons rank_static"><span class="none">순위 동일</span></span>
 <span class="none">0</span>
 </span>
 </div></td>
 <td><div class="wrap">
 <a class="image_typeAll" href="javascript:melon.link.goAlbumDetail('11127145');" title="NewJeans 'OMG'">
 <img alt="NewJeans 'OMG' - 페이지 이동" height="60" onerror="WEBPOCIMG.defaultAlbumImg(this);" src="https://cdnimg.melon.co.kr/cm2/album/images/111/27/145/11127145_20230102135733_500.jpg/melon/resize/120/quality/80/optimize" width="60"/>
 <span class="bg_album_frame"></span>
 </a>
 </div></td>
 <td><div class="wrap">
 <a class="btn button_icons type03 song_info" hr

In [32]:
# body_soup[0] : 1위 데이터 가져오기
rank=body_soup[0].find('span',class_="rank").get_text()
title=body_soup[0].find('div',class_="ellipsis rank01").get_text()
artist=body_soup[0].find('div',class_="ellipsis rank02").find('a').get_text()
ep=body_soup[0].find('div',class_="ellipsis rank03").get_text()
goods=body_soup[0].find('span',class_="cnt").get_text().replace('총건수','')

print(rank, title, artist, ep, goods.replace('\n',""))

1 
Ditto
 NewJeans 
NewJeans 'OMG'
 196,230


In [40]:
# 전체 데이터 가져오기
body_soup=soup.select('table tbody tr')

body_list=[]
for b_soup in body_soup:
    rank=b_soup.find('span',class_="rank").get_text()
    title=b_soup.find('div',class_="ellipsis rank01").get_text().replace('\n','')
    artist=b_soup.find('div',class_="ellipsis rank02").find('a').get_text()
    ep=b_soup.find('div',class_="ellipsis rank03").get_text().replace('\n','')
    goods=b_soup.find('span',class_="cnt").get_text().replace('총건수','')

    body_list.append([rank, title, artist, ep, goods.replace('\n',"")])

df=pd.DataFrame(body_list,columns=['rank', 'title','artist', 'ep', 'goods'])
df

,rank,title,artist,ep,goods
0,1,Ditto,NewJeans,NewJeans 'OMG',"196,230"
1,2,OMG,NewJeans,NewJeans 'OMG',"109,834"
2,3,Hype boy,NewJeans,NewJeans 1st EP 'New Jeans',"219,574"
3,4,사건의 지평선,윤하 (YOUNHA),YOUNHA 6th Album Repackage 'END THEORY : Final...,"270,163"
4,5,ANTIFRAGILE,LE SSERAFIM (르세라핌),ANTIFRAGILE,"139,019"
...,...,...,...,...,...
95,96,"빡 (Feat. 팔로알토 (Paloalto), 저스디스 (JUSTHIS)) (Pro...",던말릭 (DON MALIK),쇼미더머니 11 Semi Final,"16,407"
96,97,연애편지,임영웅,IM HERO,"44,236"
97,98,strawberry moon,아이유,strawberry moon,"218,223"
98,99,Beautiful Liar,몬스타엑스,REASON,"27,874"
